<a href="https://colab.research.google.com/github/ayumu111/MediaKeiensyu2/blob/mizuki/cut_picture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# モデルのロード
model = YOLO("yolo11n-pose.pt")

# 入力画像フォルダ
image_dir = "images_single_fullbody"
save_dir = "single_fullbody_pose_black_bg"
os.makedirs(save_dir, exist_ok=True)

# 連番カウンタ
counter = 1

for img_file in os.listdir(image_dir):
    if not img_file.lower().endswith((".jpg", ".png")):
        continue

    img_path = os.path.join(image_dir, img_file)
    results = model(img_path)

    if results[0].keypoints is None or len(results[0].keypoints) == 0:
        print(f"スキップ: {img_file}（キーポイントなし）")
        continue

    orig = cv2.imread(img_path)
    h, w, _ = orig.shape
    black_bg = np.zeros((h, w, 3), dtype=np.uint8)
    drawn = results[0].plot(img=black_bg.copy(), kpt_radius=5, line_width=2)

    kpts = results[0].keypoints.xy[0].cpu().numpy()
    kpts = kpts[~np.isnan(kpts).any(axis=1)]

    if len(kpts) == 0:
        print(f"スキップ: {img_file}（有効キーポイントなし）")
        continue

    x_min, y_min = kpts.min(axis=0)
    x_max, y_max = kpts.max(axis=0)

    margin = 10
    x_min = max(int(x_min) - margin, 0)
    y_min = max(int(y_min) - margin, 0)
    x_max = min(int(x_max) + margin, w)
    y_max = min(int(y_max) + margin, h)

    cropped = drawn[y_min:y_max, x_min:x_max]

    # 保存ファイル名例：0001_元ファイル名_crop.png
    save_name = f"{counter:04d}_" + os.path.splitext(img_file)[0] + "_crop.png"
    save_path = os.path.join(save_dir, save_name)

    cv2.imwrite(save_path, cropped)
    print(f"トリミング保存: {save_path}")

    counter += 1  # カウンタを増加

    plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()
